# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [1]:
!pip install langchain_cohere
!pip install langchain_text_splitters
!pip install langchain
!pip install langchain_community
!pip install langchain_core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 441.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/

In [ ]:
!pip install langchain_chroma

In [ ]:
from langchain_cohere import ChatCohere
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [ ]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

In [ ]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

  Using cached langchain_huggingface-0.1.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached Chroma-0.2.0.tar.gz (5.8 kB)
  Preparing metadata (setup.py) ... done
  Using cached langchain_groq-0.2.0-py3-none-any.whl.metadata (2.9 kB)
ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [ ]:
import pandas as pd
df = pd.read_csv('/content/Dataset.csv')

In [ ]:
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [ ]:
!pip install pypdf

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [ ]:
RecursiveCharacterTextSplitter?

In [ ]:
!pip install -qU langchain-text-splitters

In [ ]:
df2 = pd.read_csv('/content/Dataset.csv')

In [ ]:
df2

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال
...,...,...
99,القيام بأعمال الطرق قبل التنسيق مع الإدارة الم...,الغرامة المالية 5000 - 10000 ريال
100,سماح أصحاب الحيوانات بعبور حيواناتهم من غير ال...,الغرامة المالية 5000 - 10000 ريال
101,استخدام لوحات غير عائدة للمركبة.,الغرامة المالية 5000 - 10000 ريال
102,طمس رقم هيكل المركبة أو محاولة طمسه.,الغرامة المالية 5000 - 10000 ريال


In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [ ]:
df2['الغرامة'].unique()

array(['الغرامة المالية 100 - 150 ريال', 'الغرامة المالية 150 - 300 ريال',
       'الغرامة المالية 300 - 500 ريال', 'الغرامة المالية 500 - 900 ريال',
       'الغرامة المالية 1000 - 2000 ريال',
       'الغرامة المالية 3000 - 6000 ريال',
       'الغرامة المالية 5000 - 10000 ريال'], dtype=object)

In [ ]:
headers_to_split_on = [
    ("الغرامة المالية 100 - 150 ريال"),
    ("الغرامة المالية 150 - 300 ريال"),
    ("غرامة المالية 300 - 500 ريال"),
    ("الغرامة المالية 500 - 900 ريال"),
    ("الغرامة المالية 1000 - 2000 ريال"),
    ("الغرامة المالية 3000 - 6000 ريال"),
    ("'الغرامة المالية 5000 - 10000 ريال'")
]

In [ ]:
with open("my_markdown.md", 'w') as md:
  df.to_markdown(buf=md, tablefmt="grid")

In [ ]:
df.columns

Index(['المخالفة', 'الغرامة'], dtype='object')

In [ ]:
df = df.to_markdown()

In [ ]:
print(df)

|     | المخالفة                                                                                                                                      | الغرامة                           |
|----:|:----------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------|
|   0 | قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.                                                                                          | الغرامة المالية 100 - 150 ريال    |
|   1 | ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.                                                                                              | الغرامة المالية 100 - 150 ريال    |
|   2 | عدم وجود تأمين ساري للمركبة.                                                                                                                  | الغرامة المالية 100 - 150 ريال    |
|   3 | عبور المشاة للطرق من غير الأماكن المخصصة لهم.       

In [ ]:
text_spilt = RecursiveCharacterTextSplitter(
    separators=[
        "\n",
    ],
)

docs = text_spilt.split_text(df)

In [ ]:
print(docs[0])

|     | المخالفة                                                                                                                                      | الغرامة                           |
|----:|:----------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------|
|   0 | قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.                                                                                          | الغرامة المالية 100 - 150 ريال    |
|   1 | ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.                                                                                              | الغرامة المالية 100 - 150 ريال    |
|   2 | عدم وجود تأمين ساري للمركبة.                                                                                                                  | الغرامة المالية 100 - 150 ريال    |
|   3 | عبور المشاة للطرق من غير الأماكن المخصصة لهم.       

In [ ]:
print(docs[1])

|  18 | التحرك بالمركبة بسرعة عالية، بحيث تحدث الإطارات صوتاً عالياً.                                                                                   | الغرامة المالية 150 - 300 ريال    |
|  19 | التجمهر في موقع الحادث.                                                                                                                       | الغرامة المالية 150 - 300 ريال    |
|  20 | قيام السائق بتخطي أرتال المركبات أمام إشارات المرور أو نقاط الضبط الأمني باستخدام كتف الطريق أو المسار المخصص للالتفاف.                       | الغرامة المالية 150 - 300 ريال    |
|  21 | عدم توافر المتطلبات النظامية للمقطورة.                                                                                                        | الغرامة المالية 150 - 300 ريال    |
|  22 | عدم اتخاذ الاحتياطات اللازمة عند إيقاف المركبة في حالات الطوارئ على الطرق العامة.                                                             | الغرامة المالية 300 - 500 ريال    |
|  23 | رمي أجسام خارج المركبة.                           

In [ ]:
docs[0]

'|     | المخالفة                                                                                                                                      | الغرامة                           |\n|----:|:----------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------|\n|   0 | قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.                                                                                          | الغرامة المالية 100 - 150 ريال    |\n|   1 | ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.                                                                                              | الغرامة المالية 100 - 150 ريال    |\n|   2 | عدم وجود تأمين ساري للمركبة.                                                                                                                  | الغرامة المالية 100 - 150 ريال    |\n|   3 | عبور المشاة للطرق من غير الأماكن المخصصة لهم. 

In [ ]:
docs[1]

'|  18 | التحرك بالمركبة بسرعة عالية، بحيث تحدث الإطارات صوتاً عالياً.                                                                                   | الغرامة المالية 150 - 300 ريال    |\n|  19 | التجمهر في موقع الحادث.                                                                                                                       | الغرامة المالية 150 - 300 ريال    |\n|  20 | قيام السائق بتخطي أرتال المركبات أمام إشارات المرور أو نقاط الضبط الأمني باستخدام كتف الطريق أو المسار المخصص للالتفاف.                       | الغرامة المالية 150 - 300 ريال    |\n|  21 | عدم توافر المتطلبات النظامية للمقطورة.                                                                                                        | الغرامة المالية 150 - 300 ريال    |\n|  22 | عدم اتخاذ الاحتياطات اللازمة عند إيقاف المركبة في حالات الطوارئ على الطرق العامة.                                                             | الغرامة المالية 300 - 500 ريال    |\n|  23 | رمي أجسام خارج المركبة.                     

In [ ]:
docs[2]

'|  38 | عدم إعطاء الأفضلية للمركبة القادمة من اليمين عند الوصول إلى تقاطع متساوي الأفضلية في آن واحد وعندما لا يكون هناك إشارات أولوية.               | الغرامة المالية 500 - 900 ريال    |\n|  39 | عدم إعطاء الأفضلية للمركبات التي على الطريق الرئيسي في حالة عدم وجود إشارة أفضلية.                                                            | الغرامة المالية 500 - 900 ريال    |\n|  40 | عدم التقيد بإشارات رجل الأمن اليدوية عند تنظيمه للحركة وعدم إعطاء إشارته الأولوية على الإشارات الضوئية.                                       | الغرامة المالية 500 - 900 ريال    |\n|  41 | عدم إعطاء الأفضلية للمركبات التي بداخل الدوار من قبل المركبات التي خارجه في حالة عدم وجود إشارات ضوئية أو رجل أمن يوجه السير.                 | الغرامة المالية 500 - 900 ريال    |\n|  42 | قيادة المركبة داخل الأنفاق من غير إضاءة أنوارها.                                                                                              | الغرامة المالية 500 - 900 ريال    |\n|  43 | زيادة أبعاد الحمولة المنقولة لمركبات النقل الخ

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [ ]:
!pip install sentence-transformers

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

model_name = "Omartificial-Intelligence-Space/GATE-AraBert-v0"
embedding_function = HuggingFaceEmbeddings(model_name=model_name)

<ipython-input-27-466f24e95a02>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when run

In [ ]:
vector_store = Chroma.from_texts(
    texts=docs,
    embedding=embedding_function,
    persist_directory="./chroma_langchain_db"
)

retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install qdrant-client

  Using cached protobuf-5.28.2-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
Using cached protobuf-5.28.2-cp38-abi3-manylinux2014_x86_64.whl (316 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.28.2 which is incompatible.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.28.2 which is incompatible.
google-cloud-datastore 2.19.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.28.2 which is incompatible.
google-cloud-fires

In [ ]:
template = """
    {
        input: "هل تعتبر قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها مخالفة؟.",
        output: "نعم، تعتبر قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها مخالفة وفق جدول المخالفات رقم (1)، وتكون الغرامة المالية (150/100) ريال."
    }
"""
prompt = PromptTemplate(template=template)

In [ ]:
!pip install cohere

In [ ]:
!pip install cohere.Client

ERROR: Could not find a version that satisfies the requirement cohere.Client (from versions: none)
ERROR: No matching distribution found for cohere.Client


In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "XhVgCWGf3YG16xTbnIkntiw4CaYyFsw6JUbWwDjG"

llm_model = ChatCohere(cohere_api_key="XhVgCWGf3YG16xTbnIkntiw4CaYyFsw6JUbWwDjG", model = 'command-r-plus')

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm = llm_model,
    retriever=retriever
)

In [ ]:
question = "عدم إعطاء أفضلية المرور للمشاة أثناء عبورهم في المسارات المخصصة لهم.."
result = qa_chain({"query": question})
result["result"]

<ipython-input-36-59d8b1a15303>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


'الغرامة المالية 1000 - 2000 ريال'

In [ ]:
question = "اريد اكل البيتزا"
result = qa_chain({"query": question})
result["result"]

'آسف، لا يمكنني مساعدتك في ذلك.'

In [ ]:
question = "هل يوجد مخالفة في ركن السيارة"
result = qa_chain({"query": question})
result["result"]

'نعم، يوجد مخالفة لركن السيارة في أماكن غير مخصصة للوقوف، وتبلغ قيمة الغرامة المالية لهذه المخالفة 100 - 150 ريال.'

In [ ]:
question = "هل يوجد مخالفة في اكل البيتزا؟"
result = qa_chain({"query": question})
result["result"]

'لا، لا يوجد أي ذكر لمخالفة تتعلق بأكل البيتزا في جدول المخالفات المرفق.'

In [ ]:
question = "هل يوجد مخالفة في مالحقة مركبات الطوارئ أثناء استعمال المنبهات الخاصة بها.؟"
result = qa_chain({"query": question})
result["result"]

'نعم، يوجد مخالفة لملاحقة مركبات الطوارئ أثناء استعمال المنبهات الخاصة بها. رقم المخالفة هو 34، وتبلغ قيمة الغرامة المالية لها 500 - 900 ريال.'

In [ ]:
!pip install discord.py
!pip install discord.py nest_asyncio

In [ ]:
# Import necessary libraries
import nest_asyncio
import asyncio
import discord
from discord.ext import commands
from transformers import pipeline  # Assuming Hugging Face pipeline for language generation
from sentence_transformers import SentenceTransformer  # For embedding user queries

# Apply nest_asyncio to allow running asyncio in Jupyter
nest_asyncio.apply()

# Prompt for the bot token
TOKEN = input("Please enter your Discord bot token: ")

# Set up intents
intents = discord.Intents.default()
intents.message_content = True  # Required to read message content

# Create a bot instance with intents
bot = commands.Bot(command_prefix='!', intents=intents)

# Initialize vector DB connection and model (use your actual model and vector DB)
vector_db = retriever  # Replace with actual initialization

@bot.event
async def on_ready():
    print(f'{bot.user} has connected to Discord!')

# Function to retrieve relevant context and generate response
def query_rag(user_input):
    # Step 1: Convert user input to embeddings
    result = qa_chain({"query": user_input})
    return result

@bot.command(name='start')
async def start(ctx):
    await ctx.send('اهلا وسهلا، معك نظام المخالفات المرورية، كيف يمكنني مساعدتك؟')

@bot.event
async def on_message(message):
    if message.author == bot.user:
        return

    if not message.content.startswith('!'):
        # Use the RAG system to generate a response
        user_input = message.content
        response = query_rag(user_input)

        await message.channel.send(response)

    await bot.process_commands(message)

# Function to run the bot
async def run_bot():
    try:
        await bot.start(TOKEN)
    except discord.errors.LoginFailure as e:
        print(f"Error: {e}. Please check your token and try again.")

# Run the bot
asyncio.run(run_bot())


In [ ]:
# Import necessary libraries

import nest_asyncio
import asyncio
import discord
from discord.ext import commands

# Apply nest_asyncio to allow running asyncio in Jupyter
nest_asyncio.apply()

# Prompt for the bot token
TOKEN = input("Please enter your Discord bot token: ")

# Set up intents
intents = discord.Intents.default()
intents.message_content = True  # Required to read message content

# Create a bot instance with intents
bot = commands.Bot(command_prefix='!', intents=intents)

@bot.event
async def on_ready():
    print(f'{bot.user} has connected to Discord!')

# Mock query function to simulate retrieval-augmented generation
def query_rag(user_input):
    # Simulate context retrieval and response generation
    context = "This is some relevant context related to traffic violations."
    response = f"Based on your query '{user_input}', here's the advice: Always follow the traffic rules!"
    return response

@bot.command(name='start')
async def start(ctx):
    await ctx.send('Hello! I am your query-based chatbot. Send me any question, and I will provide a response.')

@bot.event
async def on_message(message):
    if message.author == bot.user:
        return

    if not message.content.startswith('!'):
        # Use the RAG system (or the mock function here) to generate a response
        user_input = message.content
        response = query_rag(user_input)

        await message.channel.send(response)

    await bot.process_commands(message)

# Function to run the bot
async def run_bot():
    try:
        await bot.start(TOKEN)
    except discord.errors.LoginFailure as e:
        print(f"Error: {e}. Please check your token and try again.")

# Run the bot
asyncio.run(run_bot())


# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.